In [1]:
from dotenv import load_dotenv
import os
from urllib.parse import quote_plus
from dotenv import load_dotenv
from sqlalchemy import create_engine
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import AzureChatOpenAI

In [2]:
load_dotenv()

True

In [3]:
azure_openai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_api_version = os.environ["OPENAI_API_VERSION"]

database_connection_string = os.environ["SQL_DATABASE_CONNECTION_STRING"]
database_password = os.environ["SQL_DATABASE_PASSWORD"]

In [ ]:
def database_connector(database_connection_string, database_password, schema): 
    try: 
        db_engine = create_engine(database_connection_string % quote_plus(database_password))
        db = SQLDatabase(db_engine, view_support=True, schema=schema)
        print(db.dialect)
        print(db.get_usable_table_names())
        db.run("select convert(varchar(25), getdate(), 120)")
        
        return db
    except Exception as e: 
        print(f"Connection to database failed. {e}")


def get_llm_client(model_name = "gpt-4", deployment = "gpt-4"): 

    azurellm = AzureChatOpenAI(
        model_name=model_name,
        deployment_name=deployment 
    )

    return azurellm

def get_sql_agent(azurellm, database):
    toolkit = SQLDatabaseToolkit(db=database, llm=azurellm)

    agent_executor = create_sql_agent(
        llm=azurellm,
        toolkit=toolkit,
        verbose=True,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        # handle_parsing_errors=True
    )

    return agent_executor

In [5]:
db = database_connector(database_connection_string=database_connection_string, database_password=database_password, schema="dbo")
azurellm = get_llm_client()
llm_agent = get_sql_agent(azurellm, db)

mssql
['curated_weather_data', 'raw_weather_data']


In [7]:
base_prompt = """You are a helpful assistant working for a energy supplier. They are keen to understand their demand forcasts to efficiently plan their operations. 
Based on the data provided, you will respond to any questions that an user might have.
For every user question which will be provided below, please return an analysis and a summary of the information you used in your analysis. 
Please dont use pleasantries of any form. Do not hallucinate, if the response is not within the data or you cannot figure it, 
return information not found."""
user_prompt = """based on the data provided in the curated table, how can I use the data to predict around what times an energy supplier can expect higher electricity demands?
"""
llm_agent.invoke(base_prompt + user_prompt)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: curated_weather_data, raw_weather_dataThe only tables I have available are related to weather data. These might not directly contain information on electricity demand but could be indirectly useful for predicting demand as weather conditions influence energy usage.

Action: sql_db_schema
Action Input: curated_weather_data, raw_weather_data
CREATE TABLE dbo.curated_weather_data (
	record_id NVARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	temperature FLOAT(53) NULL, 
	relative_humidity FLOAT(53) NULL, 
	precipitation FLOAT(53) NULL, 
	wind_direction FLOAT(53) NULL, 
	wind_speed FLOAT(53) NULL, 
	wind_gust FLOAT(53) NULL, 
	pressure FLOAT(53) NULL, 
	sun_shine FLOAT(53) NULL, 
	weather_condition_code INTEGER NULL, 
	snow FLOAT(53) NULL, 
	dew_point NVARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	time DATETIME2 NULL
)

/*
3 rows from curated_weather_data table:
record_id	temperature	

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `The available tables are focused on curated and raw weather data including temperature, humidity, precipitation, wind conditions, and atmospheric pressure, recorded over time. Weather data, particularly temperature fluctuations, might correlate with variations in electricity demand, particularly for heating or cooling.

To predict times when an energy supplier can expect higher electricity demands, one can analyze patterns and variations in weather conditions over time. For example, higher temperatures typically increase the demand for cooling systems, leading to a spike in electricity usage.

While these tables don't explicitly provide data on electricity demand, correlating historical demand data (which must be sourced elsewhere) with weather conditions could potential

In [ ]:
# handling output parser failure from SQL agent 

try:
    result = llm_agent.invoke(base_prompt + "give me a detailed summary of the data")
    print(result)
except Exception as e:
    raw_output = None
    for attr in ["output", "response", "llm_output", "text"]:
        if hasattr(e, attr):
            raw_output = getattr(e, attr)
            break

    if raw_output is None:
        import re
        match = re.search(r"Could not parse LLM output:\s*`(.+?)`", str(e), re.DOTALL)
        if match:
            raw_output = match.group(1)
        else:
            raw_output = str(e)

    print(raw_output)




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: curated_weather_data, raw_weather_dataTo further understand the data we have access to, I should fetch the schema and sample rows for the tables 'curated_weather_data' and 'raw_weather_data'. This will give an idea of the contents and structure which can help in providing a detailed summary.
Action: sql_db_schema
Action Input: curated_weather_data, raw_weather_data
CREATE TABLE dbo.curated_weather_data (
	record_id NVARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	temperature FLOAT(53) NULL, 
	relative_humidity FLOAT(53) NULL, 
	precipitation FLOAT(53) NULL, 
	wind_direction FLOAT(53) NULL, 
	wind_speed FLOAT(53) NULL, 
	wind_gust FLOAT(53) NULL, 
	pressure FLOAT(53) NULL, 
	sun_shine FLOAT(53) NULL, 
	weather_condition_code INTEGER NULL, 
	snow FLOAT(53) NULL, 
	dew_point NVARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	time DATETIME2 NULL
)

/*
3 rows from curated_weather_data tab

In [11]:
raw_output

"The database contains two tables relevant to weather data which could influence energy demand forecasting:\n\n1. **curated_weather_data**: This table seems to hold processed and refined weather data.\n   - **Columns**: record_id, temperature, relative_humidity, precipitation, wind_direction, wind_speed, wind_gust, pressure, sun_shine, weather_condition_code, snow, dew_point, time\n   - **Data Example**: Measures such as temperature, humidity, wind speed, and pressure at specific timestamps are included. No precipitation and sunshine data have been recorded in the given samples. Weather condition and dew point are also noted.\n\n2. **raw_weather_data**: This table seems to store raw weather data possibly collected directly from sensors or an external source.\n   - **Columns**: record_id, time, temp, dwpt (dew point), rhum (relative humidity), prcp (precipitation), snow, wdir (wind direction), wspd (wind speed), wpgt (wind gust), pres (pressure), tsun (sun shine), coco (weather conditio

In [ ]:
def get_data(start_datetime, end_datetime, schema="dbo"): 
       try:
              db_engine = create_engine(database_connection_string % quote_plus(database_password))
              db = SQLDatabase(db_engine, view_support=True, schema=schema)
              responses = db.run(f"""select temperature, wind_speed, sun_shine, time from [dbo].[curated_weather_data] 
                     where time>='{start_datetime}' and time<= '{end_datetime}' 
                     order by time""")

              schema = [
              "temperature",
              "wind_speed",
              "sun_shine"
              "time"]

       except Exception as e: 
              return e

       return responses.json

'[(6.8, 5.4, None, datetime.datetime(2020, 1, 2, 0, 0)), (6.8, 5.4, None, datetime.datetime(2020, 1, 2, 0, 0)), (6.8, 5.4, None, datetime.datetime(2020, 1, 2, 0, 0)), (6.8, 5.4, None, datetime.datetime(2020, 1, 2, 0, 0)), (6.8, 9.4, None, datetime.datetime(2020, 1, 2, 1, 0)), (6.8, 9.4, None, datetime.datetime(2020, 1, 2, 1, 0)), (6.8, 9.4, None, datetime.datetime(2020, 1, 2, 1, 0)), (6.8, 9.4, None, datetime.datetime(2020, 1, 2, 1, 0)), (6.8, 7.6, None, datetime.datetime(2020, 1, 2, 2, 0)), (6.8, 7.6, None, datetime.datetime(2020, 1, 2, 2, 0)), (6.8, 7.6, None, datetime.datetime(2020, 1, 2, 2, 0)), (6.8, 7.6, None, datetime.datetime(2020, 1, 2, 2, 0)), (6.8, 9.4, None, datetime.datetime(2020, 1, 2, 3, 0)), (6.8, 9.4, None, datetime.datetime(2020, 1, 2, 3, 0)), (6.8, 9.4, None, datetime.datetime(2020, 1, 2, 3, 0)), (6.8, 9.4, None, datetime.datetime(2020, 1, 2, 3, 0)), (6.8, 9.4, None, datetime.datetime(2020, 1, 2, 4, 0)), (6.8, 9.4, None, datetime.datetime(2020, 1, 2, 4, 0)), (6.8, 9.

In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional
from sqlalchemy.engine import create_engine
from sqlalchemy.exc import OperationalError
from urllib.parse import quote_plus
from fastapi.middleware.cors import CORSMiddleware
from dotenv import load_dotenv
from datetime import datetime
import os
import re
from sqlalchemy import text
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import AzureChatOpenAI

load_dotenv()

def database_connector(connection_string, db_password, schema="dbo"): 
    try: 
        db_engine = create_engine(connection_string % quote_plus(db_password))
        db = SQLDatabase(db_engine, view_support=True, schema=schema)
        print(db.dialect)
        print(db.get_usable_table_names())
        db.run("select convert(varchar(25), getdate(), 120)")
        
        return db
    except OperationalError as e:
        print(f"Connection to database failed. {e}")

database_connection_string = os.environ["SQL_DATABASE_CONNECTION_STRING"]
database_password = os.environ["SQL_DATABASE_PASSWORD"]

database_connector(database_connection_string, database_password)

mssql
['curated_weather_data', 'raw_weather_data']


In [3]:
from langchain_core.exceptions import OutputParserException  # Correct
